In [1]:
%matplotlib inline

# Redes neuronales

Dentro del deep learning, existen dos maneras de crear una red neuronal, desde cero o, lo más común, el mediante el método de *transfer learning*, este consiste en coger una red ya preentrenada para otra tarea y adaptarla a tu problema, más adelante lo explicaremos con más detalle

Vamos a ver cómo hacer esto con Pytorch

## Construyendo una red neuronal desde cero

En la librería [torch.nn](https://pytorch.org/docs/stable/nn.html) tenemos todo lo necesario para construir redes neuronales.

Para crear redes neuronales desde cero lo podemos hacer de muchas maneras, pero lo mejor es hacerlas modulares, para así poder escalar la red si hace falta, para ello, tenemos que crear una clase que herede de [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). Además tenemos que crear los métodos ``__init__`` y ``forward``.

 * Con el método ``__init__`` llamaremos al constructor de la clase de la que se hereda (``torch.nn.Module``) y además inicializaremos nuestra red con las capas que queramos que tenga
 * Con el método ``forward`` Pytorch nos permite llamar a la red como una función, aunque la hemos definido como una clase. Ahora veremos esto

Creamos una red para que aprenda a clasificar las imágenes del conjunto de datos ``CIFAR-10`` que hemos visto antes. Por recordar, ``CIFAR-10`` es un conjunto de datos de imágenes que consta de 50.000 ejemplos de entrenamiento y 10.000 ejemplos de test. Cada ejemplo contiene una imagen a color de 3x32x32 y una etiqueta asociada de una de las 10 posibles clases.

Como vamos a tener imágenes de 32x32 píxeles a color esto significa que vamos a tener matrices de 3x32x32.

De modo que lo primero que vamos a hacer es *aplanar* estas matrices, es decir, vamos a pasar estas matrices de 3x32x32 a un vector de 3072 valores (3x32x32 = 3072)

Aquí podemos ver un ejemplo de cómo se aplana una matriz de 4x4
![Aplanar](Animaciones/Aplanar_matriz.gif "Aplanar")

Una vez tenemos la imagen aplanada podemos pasarla por una red MLP (multi layer perceptron). En este caso haremos un módulo que contiene una red de dos capas

In [1]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()   # Se inicializa el módulo nn.Module
        self.flatten = nn.Flatten(start_dim=0)  # Se crea una primera capa que aplana la imagen de entrada
        self.linear_relu_stack = nn.Sequential( # Se crea una módulo de arquitectura secuencial:
            nn.Linear(3*32*32, 512),                # Se añade una primera capa lineal que está preparada 
                                                    # para que le entre un vector de 3x32x32 (3072)
                                                    # y sacará un vector de 512
            nn.ReLU(),                              # Se añade una no linealidad
            nn.Linear(512, 512),                    # Se añade una segunda capa lineal que le entran 512 
                                                    # datos y saca 512 datos
            nn.ReLU(),                              # Se añade una no linealidad
            nn.Linear(512, 10)                      # Se añade una tercera capa lineal que le entran 512 
                                                    # datos y saca un array de tamaño 10 (el número
                                                    # de etiquetas)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)                         # Se pasa la imagen por la capa de aplanado
        x.unsqueeze_(0)                             # Se añade una dimensión para que todo cuadre
        logits = self.linear_relu_stack(x)          # Se pasa el vector resultante por la red
        probs = self.softmax(logits)
        return probs

Hemos definido la red neuronal, es decir, le hemos dicho a Pytorch cómo queremos que sea, ahora hace falta instanciarla, es decir, construirla

In [2]:
model = NeuralNetwork()

Una vez tenemos la red neuronal creada podemos imprimir su estructura

In [3]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=0, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)


Vamos a probar que nuestra red funciona, creamos una matriz de números aleatorios de 3x32x32 y se lo pasamos a la 
red

In [4]:
import torch

input = torch.rand(3, 32, 32)
output = model(input)

output

tensor([[0.1172, 0.1049, 0.0977, 0.0918, 0.0826, 0.1161, 0.0987, 0.0882, 0.0999,
         0.1029]], grad_fn=<SoftmaxBackward0>)

La red devuelve un vector con las probabilidades de cada clase. La posición donde se encuentra el máximo de estos valores corresponde a la clase que la red cree que pertenece la matriz que le hemos metido

In [5]:
posicion = output.argmax(1)

posicion.item()

0

Ya sabemos la posición, veamos por casualidad qué valor tiene la salida en esa posición

In [69]:
output[posicion]

IndexError: index 7 is out of bounds for dimension 0 with size 1

Obtenemos un error, esto es porque la salida no es un vector, sino una matriz de tamaño 1x10

In [6]:
output.shape

torch.Size([1, 10])

Así que usamos el método ``squeeze()`` que ya vimos en la clase anterior para quitar la dimensión que tiene valor 1

In [7]:
output.squeeze().shape, output.squeeze()[posicion]

(torch.Size([10]), tensor([0.1172], grad_fn=<IndexBackward0>))

Ya vemos que con el método ``squeeze()`` tenemos un vector, y que podemos ver la salida en la posición con el valor máximo

Hacemos un diccionario con todas las posibles clases para ver qué ha predicho la red

In [8]:
labels_map = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck",
}

labels_map[posicion.item()]

'airplane'

### Parámetros del modelo

Puede que nos interese ver los parámetros de la red neuronal. En este ejemplo iteramos por todas las capas e imprimimos su tamaño y sus valores

In [9]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=0, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 3072]) | Values : tensor([[ 0.0109,  0.0023, -0.0139,  ..., -0.0058, -0.0054,  0.0146],
        [ 0.0104, -0.0014, -0.0066,  ...,  0.0008, -0.0018,  0.0049]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0121, -0.0141], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0166, -0.0052, -0.0053,  ...,  0.0298, -0.0098, -0.0402],
        [ 0.0250,  0.0438,  0.0184,  ...,  0.0326, -0.0438, -0.0310]],
       grad_fn=<SliceBackward0>) 

La

## Construyendo una red neuronal mediante *transfer learning*

Lo anterior está muy bien, podemos crearnos ua red neuronal a nuestro gusto, con la arquitectura que queramos. Esto es una de las cualidades de Pytorch, que podemos construir muy facilmente cualquier estructura de red. 
En el ejemplo anterior hemos construido una red neuronal secuencial, es decir la salida de capa entrada era la entrada de la siguiente capa, pero muy facilmente en el método ``forward`` podiamos haber hecho las conexiones que nos hubiera dado la gana

Pero hacer una red neuronal desde cero tiene el problema de que los parámetros se crean aleatoriamente, por lo que hay que entrenarla desde cero.

Por ello, en la mayoría de los casos lo que se hace es aprovechar el *conocimiento* de otras redes neuronales. Redes entrenadas para otros propósitos, pero que no pueden valer.

Existe una competición llamada ``ImageNet``, en la cual se tiene que conseguir clasificar imágenes de una base de datos con más de 14 millones de imágenes, las cuales pertenecen a 1000 categorías distintas.

A esta competición se presentan grupos de investigación de todo el mundo para poner a prueba las redes neuronales que han desarrollado. Por suerte estas redes ya entrenadas para esta competición están disponibles para todo el mundo.
Por lo que nosotros podemos coger una de estas redes y adaptarlas para nuestro problema

Por ejemplo, una de estas redes preentrenadas es la ``Resnet18``. Hay varios tipos de Resnet, el número es la cantidad de capas, por lo que en nuestro caso solo tien 18 capas. Hay ``Resnet34``, con 34 capas; ``Resnet50``, con 50 capas y varias más hasta la ``Resnet152``, con 152 capas.

Nos quedamos con las ``Resnet18``, porque es un modelo pequeño y para lo que queremos hacer ahora nos vale.

Pytorch tiene todas estas `Resnet` y otras muchas más [redes de visión](https://pytorch.org/vision/stable/models.html) disponibles para descargar

 > Importante: Para descargarse la red preentrenada hay que poner el parámetro `pretrained` a True

In [10]:
from torchvision import models

model = models.resnet18(pretrained=True)

model

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/mfnunez/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 59.2MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Al imprimir la red podemos ver todas las capas que tiene, pero en especial podemos ver una capa llamada ``fc`` que es una capa lineal que tiene 512 características de entrada y 1000 características de salida.

```(fc): Linear(in_features=512, out_features=1000, bias=True)```

No es el objetivo de esta clase explicar la ``Resnet18``, por lo que solo diremos que a groso modo, ``Resnet18`` tiene un montón de capas entrenadas para el conjunto de datos de ``ImageNet``, por eso, en la última capa hay 1000 características de salida, correspondientes a las 1000 clases de ``ImageNet``.

En la parte de visión se explicará en detalle este tipo de redes, pero a groso modo, las primeras capas de la red son capaces de reconocer formas simples como lineas o esquinas. Y a medida que vamos adentrándonos en las capas, la red es capad de reconocer formas más complejas, como ojos, caras, etc. Hasta llegar a la última capa donde es capaz de reconocer los objetos de ``ImageNet``, como coches, aviones, perros, ...

Lo que vamos a hacer nosotros es eliminar la última capa, la llamada ``fc``, que tiene 1000 neuronas, para las 1000 clases de ``ImageNet`` y la vamos a sustituir por otra capa lineal, pero en este caso de 10 neuronas, para las 10 clases de nuestro problema

In [11]:
model.fc = nn.Linear(512, 10)

Si ahora imprimimos el modelo veremos que la última capa ha cambiado y ahora solo tiene 10 neuronas a la salida

In [12]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Vamos a probar que nuestra red funciona, creamos una matriz de números aleatorios de 64x3x32x32 y se lo pasamos a la red. Ahora se añade la dimensión inicial con valor 64, porque la red espera un tensor en el que el primer elemento es el numero de baches ,luego el número de canales y por último el alto y el ancho. De modo que hacemos como que tenemos un bach de 64 imágenes

In [13]:
import torch

input = torch.rand(64, 3, 32, 32)
output = model(input)

output[0]

tensor([ 1.3794,  0.0895,  1.2486, -0.4062, -0.0686,  0.4299,  0.8012,  0.6490,
        -0.3423,  1.0338], grad_fn=<SelectBackward0>)

La red devuelve un vector con los valores de cada clase.

Para obtener las probabilidades de cada clase, a la salida le tenemos que pasar por la función `softmax`

La posición donde se encuentra la probabilidad máxima de estos valores corresponde a la clase que la red cree que pertenece la matriz que le hemos metido

In [14]:
probabilidades = output.softmax(dim=1)

posiciones = probabilidades.argmax(1)

posicion = posiciones[0].item()

posicion

0

Ya sabemos la posición, veamos por casualidad qué valor tiene la salida en esa posición

In [15]:
output[0][posicion]

tensor(1.3794, grad_fn=<SelectBackward0>)

Hacemos un diccionario con todas las posibles clases para ver qué ha predicho la red

In [16]:
labels_map = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck",
}

labels_map[posicion]

'airplane'